# 05 - Patient Risk Clustering
## Discovering Natural Patient Risk Groups for DDI Analysis

---

**Objective:** Use unsupervised machine learning (clustering) to discover natural groupings of patients based on their DDI risk profiles.

**Approach:**
- K-means clustering on patient-level features
- Determine optimal number of clusters using elbow method and silhouette analysis
- Characterize and name clusters based on clinical characteristics
- Validate cluster quality statistically and clinically
- Save clustered patient data for downstream analysis

**Expected Outcomes:**
- 3-5 distinct patient risk clusters (e.g., "Elderly High-Risk Polypharmacy", "Young Low-Risk")
- Patient features enriched with cluster labels
- Visualizations showing cluster separation
- Foundation for targeted clinical interventions

**References:**
- See `../docs/CLUSTERING_AND_ANALYSIS_GUIDE.md` for detailed methodology
- See `../docs/FEATURE_ENGINEERING_GUIDE.md` for feature descriptions

## Part 1: Setup and Configuration

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import logging
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Clustering
from sklearn.cluster import KMeans, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage

# Dimensionality reduction for visualization
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Metrics and validation
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    silhouette_samples
)

# Statistical testing
from scipy.stats import f_oneway

# S3/MinIO access
import s3fs
import boto3

# Configuration
from config import *

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

logger.info("Setup complete - imports successful")

## Part 2: Load Patient Features

In [ ]:
# Initialize S3 filesystem
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': f'http://{MINIO_ENDPOINT}'}
)

# Load patient features from v3_features tier
input_uri = f"s3://{DEST_BUCKET}/v3_features/ddi/patients_features.parquet"
logger.info(f"Loading patient features from: {input_uri}")

try:
    patient_features = pd.read_parquet(input_uri, filesystem=fs)
    logger.info(f"Loaded {len(patient_features)} patient records")
    logger.info(f"Features available: {len(patient_features.columns)} columns")
except Exception as e:
    logger.error(f"Error loading patient features: {e}")
    raise

# Display basic info
print("\n=== Patient Features Dataset ===")
print(f"Number of patients: {len(patient_features)}")
print(f"Number of features: {len(patient_features.columns)}")
print("\nFirst few rows:")
patient_features.head()

In [ ]:
# Examine feature data types and completeness
print("\n=== Feature Info ===")
patient_features.info()

print("\n=== Missing Values ===")
missing = patient_features.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values detected")

## Part 3: Feature Selection for Clustering

**Important:** Not all features should be used for clustering. We select features based on:
- **Clinical relevance** - Features that matter for risk stratification
- **Variability** - Features that differ across patients
- **Independence** - Avoid highly correlated features

We'll use features that capture:
1. Demographics (Age, Gender)
2. Medication profile (unique medications, diversity, temporal patterns)
3. DDI risk (counts, severity, density)
4. Care complexity (source diversity)

In [ ]:
# Define clustering features
# Excludes: PatientSID (ID), dates (temporal), derived binary flags that are combinations
clustering_features = [
    # Demographics
    'Age',
    'Gender',  # Will need encoding

    # Medication profile
    'unique_medications',
    'medication_diversity',
    'avg_medications_per_day',

    # Temporal
    'medication_timespan_days',

    # Source system
    'source_diversity',

    # DDI risk metrics
    'ddi_pair_count',
    'ddi_severity_Moderate',  # Changed: Only Moderate severity exists
    'ddi_severity_Low',       # Added: Low severity is available
    'total_ddi_risk_score',
    'ddi_density',
    'max_severity_level'
]

logger.info(f"Selected {len(clustering_features)} features for clustering")
print("\n=== Clustering Features ===")
for i, feat in enumerate(clustering_features, 1):
    print(f"{i:2d}. {feat}")

# Verify all features exist
missing_features = [f for f in clustering_features if f not in patient_features.columns]
if missing_features:
    logger.error(f"Missing features: {missing_features}")
    raise ValueError(f"Features not found in dataset: {missing_features}")
else:
    logger.info("All clustering features present in dataset")

## Part 4: Feature Preprocessing

### Why Preprocessing Matters:
1. **Categorical encoding**: Convert Gender to numeric (M=1, F=0, U=-1)
2. **Missing values**: Handle any NaN values
3. **Scaling**: Standardize features so each contributes equally to distance calculations

Without scaling, features with larger numeric ranges would dominate clustering.

In [ ]:
# Extract clustering features
X = patient_features[clustering_features].copy()

print("=== Pre-processing Steps ===")
print(f"\nOriginal shape: {X.shape}")

# Step 1: Encode Gender (categorical variable)
print("\n1. Encoding Gender...")
print(f"   Gender distribution before encoding:")
print(f"   {X['Gender'].value_counts()}")

gender_mapping = {'M': 1, 'F': 0, 'U': -1}
X['Gender'] = X['Gender'].map(gender_mapping)
print(f"   Encoding: M=1, F=0, U=-1")

# Step 2: Check for missing values
print("\n2. Checking for missing values...")
missing_counts = X.isnull().sum()
if missing_counts.sum() > 0:
    print("   Missing values found:")
    print(missing_counts[missing_counts > 0])
    
    # Simple imputation: use median for numeric features
    print("   Imputing with median values...")
    X = X.fillna(X.median())
    logger.info("Missing values imputed with median")
else:
    print("   No missing values detected")

# Step 3: Scale features
print("\n3. Scaling features (StandardScaler)...")
print("   Before scaling - feature ranges:")
print(X.describe().loc[['min', 'max']].T)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

print("\n   After scaling - all features have mean≈0, std≈1:")
print(X_scaled_df.describe().loc[['mean', 'std']].T)

logger.info(f"Preprocessing complete - final shape: {X_scaled.shape}")
print(f"\nFinal preprocessing result: {X_scaled.shape[0]} samples, {X_scaled.shape[1]} features")

## Part 5: Determine Optimal Number of Clusters (K)

**Challenge:** K-means requires specifying the number of clusters upfront.

**Solution:** Use two methods to find optimal K:
1. **Elbow Method** - Look for "elbow" in inertia plot
2. **Silhouette Score** - Find K with best cluster separation

We'll test K from 2 to 10 and compare results.

In [ ]:
# Test range of K values
K_range = range(2, 11)
inertias = []
silhouette_scores = []
davies_bouldin_scores = []
calinski_harabasz_scores = []

logger.info(f"Testing K from {min(K_range)} to {max(K_range)}...")

for k in K_range:
    # Fit K-means
    kmeans = KMeans(
        n_clusters=k,
        random_state=RANDOM_STATE,
        n_init=10,
        max_iter=300
    )
    labels = kmeans.fit_predict(X_scaled)
    
    # Calculate metrics
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, labels))
    davies_bouldin_scores.append(davies_bouldin_score(X_scaled, labels))
    calinski_harabasz_scores.append(calinski_harabasz_score(X_scaled, labels))
    
    logger.info(f"K={k}: inertia={kmeans.inertia_:.2f}, silhouette={silhouette_scores[-1]:.3f}")

logger.info("Cluster evaluation complete")

In [ ]:
# Visualize cluster quality metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Elbow Method - Inertia
axes[0, 0].plot(K_range, inertias, marker='o', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0, 0].set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
axes[0, 0].set_title('Elbow Method for Optimal K', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(K_range)

# 2. Silhouette Score (Higher is better)
axes[0, 1].plot(K_range, silhouette_scores, marker='o', linewidth=2, markersize=8, color='green')
axes[0, 1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0, 1].set_ylabel('Silhouette Score', fontsize=12)
axes[0, 1].set_title('Silhouette Score vs K (Higher is Better)', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xticks(K_range)
axes[0, 1].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Good separation (>0.5)')
axes[0, 1].legend()

# Mark best silhouette score
best_k_silhouette = K_range[np.argmax(silhouette_scores)]
best_silhouette = max(silhouette_scores)
axes[0, 1].annotate(
    f'Best K={best_k_silhouette}\n({best_silhouette:.3f})',
    xy=(best_k_silhouette, best_silhouette),
    xytext=(best_k_silhouette + 1, best_silhouette - 0.05),
    arrowprops=dict(arrowstyle='->', color='red'),
    fontsize=10,
    fontweight='bold'
)

# 3. Davies-Bouldin Index (Lower is better)
axes[1, 0].plot(K_range, davies_bouldin_scores, marker='o', linewidth=2, markersize=8, color='orange')
axes[1, 0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1, 0].set_ylabel('Davies-Bouldin Index', fontsize=12)
axes[1, 0].set_title('Davies-Bouldin Index vs K (Lower is Better)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xticks(K_range)

# 4. Calinski-Harabasz Index (Higher is better)
axes[1, 1].plot(K_range, calinski_harabasz_scores, marker='o', linewidth=2, markersize=8, color='purple')
axes[1, 1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1, 1].set_ylabel('Calinski-Harabasz Index', fontsize=12)
axes[1, 1].set_title('Calinski-Harabasz Index vs K (Higher is Better)', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xticks(K_range)

plt.tight_layout()
plt.show()

# Print recommendations
print("\n=== Cluster Quality Metrics Summary ===")
print(f"\nBest K by Silhouette Score: {best_k_silhouette} (score: {best_silhouette:.3f})")
print(f"Best K by Davies-Bouldin: {K_range[np.argmin(davies_bouldin_scores)]} (score: {min(davies_bouldin_scores):.3f})")
print(f"Best K by Calinski-Harabasz: {K_range[np.argmax(calinski_harabasz_scores)]} (score: {max(calinski_harabasz_scores):.1f})")

In [ ]:
# Select optimal K based on analysis
# Prioritize silhouette score with clinical sensibility check
optimal_k = best_k_silhouette

print(f"\n{'='*60}")
print(f"SELECTED OPTIMAL K = {optimal_k}")
print(f"{'='*60}")
print(f"\nRationale:")
print(f"  - Silhouette score: {silhouette_scores[optimal_k-2]:.3f} (good separation)")
print(f"  - Clinically interpretable: {optimal_k} patient groups")
print(f"  - Actionable: Can define distinct interventions for each group")

logger.info(f"Optimal K selected: {optimal_k}")

## Part 6: Apply K-Means Clustering

Now we'll apply K-means with the optimal K and assign cluster labels to all patients.

In [ ]:
# Fit final K-means model
logger.info(f"Fitting K-means with K={optimal_k}...")

kmeans_final = KMeans(
    n_clusters=optimal_k,
    random_state=RANDOM_STATE,
    n_init=10,
    max_iter=300
)

cluster_labels = kmeans_final.fit_predict(X_scaled)

# Add cluster labels to original patient features
patient_features['cluster'] = cluster_labels

# Calculate final metrics
final_silhouette = silhouette_score(X_scaled, cluster_labels)
final_inertia = kmeans_final.inertia_

logger.info(f"Clustering complete - Silhouette: {final_silhouette:.3f}, Inertia: {final_inertia:.2f}")

print(f"\n=== Clustering Results ===")
print(f"Number of clusters: {optimal_k}")
print(f"Silhouette score: {final_silhouette:.3f}")
print(f"Inertia: {final_inertia:.2f}")
print(f"\nCluster sizes:")
print(patient_features['cluster'].value_counts().sort_index())

## Part 7: Cluster Characterization

For each cluster, we'll calculate:
- Mean and median of key features
- Patient count and percentage
- Clinical indicators (elderly, polypharmacy, high risk)

This helps us understand what makes each cluster unique and clinically meaningful.

In [ ]:
# Define key features for characterization
characterization_features = [
    'Age',
    'unique_medications',
    'ddi_pair_count',
    'total_ddi_risk_score',
    'ddi_density',
    'medication_timespan_days',
    'source_diversity',
    'max_severity_level'
]

# Calculate cluster statistics
cluster_summary = patient_features.groupby('cluster').agg({
    'PatientSID': 'count',  # Number of patients
    'Age': ['mean', 'median'],
    'unique_medications': ['mean', 'median'],
    'ddi_pair_count': ['mean', 'median'],
    'total_ddi_risk_score': ['mean', 'median'],
    'ddi_density': ['mean', 'median'],
    'is_polypharmacy': 'sum',  # Count of polypharmacy patients
    'is_high_ddi_risk': 'sum',  # Count of high-risk patients
    'IsElderly': 'sum'  # Count of elderly patients
})

# Flatten column names
cluster_summary.columns = ['_'.join(col).strip() if col[1] else col[0] for col in cluster_summary.columns.values]
cluster_summary = cluster_summary.rename(columns={'PatientSID_count': 'patient_count'})

# Calculate percentages
total_patients = len(patient_features)
cluster_summary['percentage'] = (cluster_summary['patient_count'] / total_patients * 100).round(1)
cluster_summary['polypharmacy_rate'] = (cluster_summary['is_polypharmacy_sum'] / cluster_summary['patient_count'] * 100).round(1)
cluster_summary['high_risk_rate'] = (cluster_summary['is_high_ddi_risk_sum'] / cluster_summary['patient_count'] * 100).round(1)
cluster_summary['elderly_rate'] = (cluster_summary['IsElderly_sum'] / cluster_summary['patient_count'] * 100).round(1)

print("\n=== Cluster Characterization Summary ===")
print(cluster_summary[[
    'patient_count', 'percentage',
    'Age_mean', 'unique_medications_mean', 'ddi_pair_count_mean',
    'total_ddi_risk_score_mean', 'elderly_rate', 'polypharmacy_rate', 'high_risk_rate'
]].round(2))

In [ ]:
# Name clusters based on characteristics
# This is done manually based on the cluster summary
# You'll need to review the summary above and assign meaningful names

# Example cluster naming (adjust based on actual results)
cluster_names = {}

# Analyze each cluster to assign names
for cluster_id in range(optimal_k):
    row = cluster_summary.loc[cluster_id]
    
    # Determine characteristics
    avg_age = row['Age_mean']
    avg_meds = row['unique_medications_mean']
    avg_ddi = row['ddi_pair_count_mean']
    elderly_pct = row['elderly_rate']
    polypharm_pct = row['polypharmacy_rate']
    high_risk_pct = row['high_risk_rate']
    
    # Name based on characteristics
    if elderly_pct > 70 and polypharm_pct > 70 and high_risk_pct > 70:
        name = "Elderly High-Risk Polypharmacy"
    elif avg_age < 50 and avg_ddi < 2 and high_risk_pct < 30:
        name = "Young Low-Risk"
    elif avg_age >= 50 and avg_age < 65 and polypharm_pct > 40:
        name = "Middle-Age Moderate Complexity"
    elif avg_ddi > 3 and high_risk_pct > 60:
        name = "High DDI Burden"
    else:
        name = f"Cluster {cluster_id}"
    
    cluster_names[cluster_id] = name
    
    print(f"\nCluster {cluster_id}: {name}")
    print(f"  - Age: {avg_age:.1f} years ({elderly_pct:.0f}% elderly)")
    print(f"  - Medications: {avg_meds:.1f} unique ({polypharm_pct:.0f}% polypharmacy)")
    print(f"  - DDI pairs: {avg_ddi:.1f} ({high_risk_pct:.0f}% high risk)")
    print(f"  - Patients: {int(row['patient_count'])} ({row['percentage']:.1f}%)")

# Add cluster names to patient features
patient_features['cluster_name'] = patient_features['cluster'].map(cluster_names)

logger.info(f"Clusters named: {list(cluster_names.values())}")

## Part 8: Cluster Visualization

We'll use PCA (Principal Component Analysis) to reduce our 14-dimensional feature space to 2D for visualization.

In [ ]:
# Apply PCA for 2D visualization
pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)

# Create PCA scatter plot
plt.figure(figsize=(12, 8))

# Plot each cluster with different color
scatter = plt.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=cluster_labels,
    cmap='viridis',
    s=100,
    alpha=0.6,
    edgecolors='black',
    linewidth=0.5
)

# Plot cluster centroids
centroids_pca = pca.transform(kmeans_final.cluster_centers_)
plt.scatter(
    centroids_pca[:, 0],
    centroids_pca[:, 1],
    c='red',
    s=300,
    alpha=0.8,
    marker='X',
    edgecolors='black',
    linewidth=2,
    label='Centroids'
)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('Patient Clusters (PCA 2D Projection)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nPCA Variance Explained: {pca.explained_variance_ratio_.sum():.1%} total")
print(f"  PC1: {pca.explained_variance_ratio_[0]:.1%}")
print(f"  PC2: {pca.explained_variance_ratio_[1]:.1%}")

In [ ]:
# Create heatmap of cluster characteristics
# Select key features for heatmap
heatmap_features = [
    'Age_mean',
    'unique_medications_mean',
    'ddi_pair_count_mean',
    'total_ddi_risk_score_mean',
    'ddi_density_mean',
    'elderly_rate',
    'polypharmacy_rate',
    'high_risk_rate'
]

# Create heatmap data
heatmap_data = cluster_summary[heatmap_features].T
heatmap_data.columns = [cluster_names[i] for i in heatmap_data.columns]

# Normalize for better visualization (0-1 scale)
from sklearn.preprocessing import MinMaxScaler
scaler_hm = MinMaxScaler()
heatmap_normalized = pd.DataFrame(
    scaler_hm.fit_transform(heatmap_data),
    columns=heatmap_data.columns,
    index=heatmap_data.index
)

# Rename index for readability
index_labels = [
    'Mean Age',
    'Mean Medications',
    'Mean DDI Pairs',
    'Mean Risk Score',
    'Mean DDI Density',
    'Elderly Rate (%)',
    'Polypharmacy Rate (%)',
    'High Risk Rate (%)'
]
heatmap_normalized.index = index_labels

# Create heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    heatmap_normalized,
    annot=heatmap_data.round(1),
    fmt='.1f',
    cmap='YlOrRd',
    cbar_kws={'label': 'Normalized Value (0-1)'},
    linewidths=0.5,
    linecolor='gray'
)
plt.title('Cluster Characteristics Heatmap', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Create bar charts comparing clusters on key metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Prepare data with cluster names
plot_data = cluster_summary.copy()
plot_data['cluster_name'] = [cluster_names[i] for i in plot_data.index]

# 1. Age comparison
plot_data.plot(x='cluster_name', y='Age_mean', kind='bar', ax=axes[0, 0], color='steelblue', legend=False)
axes[0, 0].set_title('Mean Age by Cluster', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Cluster', fontsize=10)
axes[0, 0].set_ylabel('Mean Age (years)', fontsize=10)
axes[0, 0].set_xticklabels(plot_data['cluster_name'], rotation=45, ha='right')

# 2. Medication count comparison
plot_data.plot(x='cluster_name', y='unique_medications_mean', kind='bar', ax=axes[0, 1], color='green', legend=False)
axes[0, 1].set_title('Mean Unique Medications by Cluster', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Cluster', fontsize=10)
axes[0, 1].set_ylabel('Mean Unique Medications', fontsize=10)
axes[0, 1].set_xticklabels(plot_data['cluster_name'], rotation=45, ha='right')

# 3. DDI risk score comparison
plot_data.plot(x='cluster_name', y='total_ddi_risk_score_mean', kind='bar', ax=axes[1, 0], color='orange', legend=False)
axes[1, 0].set_title('Mean DDI Risk Score by Cluster', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Cluster', fontsize=10)
axes[1, 0].set_ylabel('Mean Risk Score', fontsize=10)
axes[1, 0].set_xticklabels(plot_data['cluster_name'], rotation=45, ha='right')

# 4. High risk rate comparison
plot_data.plot(x='cluster_name', y='high_risk_rate', kind='bar', ax=axes[1, 1], color='red', legend=False)
axes[1, 1].set_title('High DDI Risk Rate by Cluster', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Cluster', fontsize=10)
axes[1, 1].set_ylabel('High Risk Rate (%)', fontsize=10)
axes[1, 1].set_xticklabels(plot_data['cluster_name'], rotation=45, ha='right')

plt.tight_layout()
plt.show()

## Part 9: Statistical Validation

We'll use ANOVA to test if clusters have significantly different mean DDI risk scores.

In [ ]:
# Perform ANOVA to test if clusters differ significantly
cluster_groups = [
    patient_features[patient_features['cluster'] == i]['total_ddi_risk_score'].values
    for i in range(optimal_k)
]

f_stat, p_value = f_oneway(*cluster_groups)

print("\n=== Statistical Validation (ANOVA) ===")
print(f"Null Hypothesis: All clusters have the same mean DDI risk score")
print(f"\nF-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.6f}")

if p_value < 0.05:
    print(f"\n✅ RESULT: Clusters have significantly different mean DDI risk scores (p < 0.05)")
    print(f"   Conclusion: The clustering successfully identified distinct patient risk groups")
else:
    print(f"\n❌ RESULT: No significant difference between clusters (p >= 0.05)")
    print(f"   Conclusion: Consider different K or feature selection")

logger.info(f"ANOVA: F={f_stat:.4f}, p={p_value:.6f}")

In [ ]:
# Calculate per-sample silhouette scores
silhouette_vals = silhouette_samples(X_scaled, cluster_labels)
patient_features['silhouette_score'] = silhouette_vals

# Analyze silhouette scores by cluster
print("\n=== Silhouette Scores by Cluster ===")
for cluster_id in range(optimal_k):
    cluster_silhouette_vals = silhouette_vals[cluster_labels == cluster_id]
    print(f"\n{cluster_names[cluster_id]}:")
    print(f"  Mean silhouette: {cluster_silhouette_vals.mean():.3f}")
    print(f"  Min silhouette: {cluster_silhouette_vals.min():.3f}")
    print(f"  Max silhouette: {cluster_silhouette_vals.max():.3f}")
    
    # Count poorly assigned patients (negative silhouette)
    poorly_assigned = (cluster_silhouette_vals < 0).sum()
    if poorly_assigned > 0:
        print(f"  ⚠️  {poorly_assigned} patients may be misclassified (silhouette < 0)")

## Part 10: Save Clustered Data

Save the patient features enriched with cluster assignments for use in 06_analysis.ipynb.

In [ ]:
# Save clustered patient data
output_uri = f"s3://{DEST_BUCKET}/v3_features/ddi/patients_features_clustered.parquet"
logger.info(f"Saving clustered patient data to: {output_uri}")

try:
    patient_features.to_parquet(output_uri, filesystem=fs, index=False)
    logger.info(f"Successfully saved {len(patient_features)} patient records with cluster labels")
except Exception as e:
    logger.error(f"Error saving clustered data: {e}")
    raise

print(f"\n✅ Clustered patient data saved to: {output_uri}")
print(f"   Records: {len(patient_features)}")
print(f"   Columns: {len(patient_features.columns)}")
print(f"   New columns: cluster, cluster_name, silhouette_score")

In [ ]:
# Save cluster summary statistics
cluster_summary_path = "cluster_summary.csv"
cluster_summary.to_csv(cluster_summary_path)
logger.info(f"Cluster summary saved to: {cluster_summary_path}")

print(f"\n✅ Cluster summary saved to: {cluster_summary_path}")

In [ ]:
# Save clustering metrics
import json

metrics = {
    'optimal_k': int(optimal_k),
    'silhouette_score': float(final_silhouette),
    'inertia': float(final_inertia),
    'davies_bouldin_score': float(davies_bouldin_score(X_scaled, cluster_labels)),
    'calinski_harabasz_score': float(calinski_harabasz_score(X_scaled, cluster_labels)),
    'anova_f_statistic': float(f_stat),
    'anova_p_value': float(p_value),
    'pca_variance_explained': float(pca.explained_variance_ratio_.sum()),
    'cluster_names': cluster_names,
    'timestamp': datetime.now().isoformat()
}

metrics_path = "cluster_metrics.json"
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)

logger.info(f"Clustering metrics saved to: {metrics_path}")
print(f"\n✅ Clustering metrics saved to: {metrics_path}")

## Part 11: Summary and Clinical Interpretation

In [ ]:
print("\n" + "="*80)
print(" "*20 + "CLUSTERING ANALYSIS COMPLETE")
print("="*80)

print(f"\n📊 CLUSTERING RESULTS:")
print(f"   • Number of clusters identified: {optimal_k}")
print(f"   • Total patients clustered: {len(patient_features)}")
print(f"   • Silhouette score: {final_silhouette:.3f} (>0.5 = good separation)")
print(f"   • Statistical validation: {'✅ Significant' if p_value < 0.05 else '❌ Not significant'} (p={p_value:.6f})")

print(f"\n👥 IDENTIFIED PATIENT GROUPS:")
for cluster_id in range(optimal_k):
    name = cluster_names[cluster_id]
    count = cluster_summary.loc[cluster_id, 'patient_count']
    pct = cluster_summary.loc[cluster_id, 'percentage']
    avg_age = cluster_summary.loc[cluster_id, 'Age_mean']
    avg_meds = cluster_summary.loc[cluster_id, 'unique_medications_mean']
    avg_risk = cluster_summary.loc[cluster_id, 'total_ddi_risk_score_mean']
    
    print(f"\n   {cluster_id+1}. {name}")
    print(f"      Patients: {int(count)} ({pct:.1f}%)")
    print(f"      Avg Age: {avg_age:.1f} years")
    print(f"      Avg Medications: {avg_meds:.1f}")
    print(f"      Avg DDI Risk Score: {avg_risk:.1f}")

print(f"\n💾 OUTPUTS CREATED:")
print(f"   • {output_uri}")
print(f"   • {cluster_summary_path}")
print(f"   • {metrics_path}")

print(f"\n📈 NEXT STEPS:")
print(f"   1. Review cluster names and adjust if needed")
print(f"   2. Proceed to 06_analysis.ipynb for deep-dive analysis")
print(f"   3. Use cluster labels to develop targeted interventions")
print(f"   4. Consider cluster-specific predictive models")

print("\n" + "="*80)
logger.info("Clustering notebook execution complete")